In [ ]:
from pynq import Overlay
import time
ol = Overlay('matrix.bit')

In [ ]:
#ol?

In [ ]:
dma = ol.axi_dma_0

In [ ]:
#dma?

In [ ]:
data_send = dma.sendchannel
data_recv = dma.recvchannel

In [ ]:
import numpy as np
import matplotlib as plt
import random

In [ ]:
np.random.seed(2)
data_A = np.random.uniform(low=0, high=10, size=(100, 100)).astype(float)
data_B = np.random.uniform(low=0, high=10, size=(100, 100)).astype(float)

In [ ]:
data_A

array([[4.35994902, 0.25926232, 5.49662478, ..., 6.61481327, 1.70477133,
        8.81652236],
       [7.7800816 , 1.33954208, 8.68916626, ..., 5.7027421 , 9.16714612,
        7.02264241],
       [5.01216452, 5.06086834, 2.18820786, ..., 4.3666988 , 0.15056134,
        8.92905883],
       ...,
       [8.47501085, 1.31358627, 7.48304695, ..., 6.13864418, 3.01381357,
        8.35725715],
       [5.90558724, 2.81559236, 4.71002749, ..., 6.55505965, 2.18741538,
        8.92183124],
       [7.20691133, 8.85091351, 7.88750645, ..., 7.34176132, 7.73163186,
        1.73273328]])

In [ ]:
data_B

array([[0.45209358, 2.60834748, 8.56704315, ..., 2.37114778, 9.92721777,
        8.10776828],
       [1.94367587, 8.3472529 , 1.51141799, ..., 9.86539894, 2.7422922 ,
        3.71473337],
       [3.30308581, 3.61424772, 1.11340129, ..., 9.76225661, 8.75260893,
        8.83062626],
       ...,
       [4.59408327, 3.78022009, 2.15007284, ..., 5.41924595, 8.7485823 ,
        9.21298658],
       [1.15957171, 1.2703291 , 3.81098302, ..., 4.96930051, 3.38287824,
        6.98791335],
       [8.92869972, 5.85914667, 3.55248482, ..., 0.08168716, 8.70860167,
        0.38990867]])

In [ ]:
def multiply(matrix_a, matrix_b):
    result_matrix = np.zeros((100,100))
    for i in range(100):
        for j in range(100):
            for k in range(100):
                result_matrix[i][j] += matrix_a[i][k] * matrix_b[k][j]

    return result_matrix

In [ ]:
start_time = time.time()
multiply(data_A, data_B) #software
end_time = time.time()
print(end_time - start_time)

19.647571802139282


In [ ]:
from pynq import allocate

In [ ]:
#allocate?

In [ ]:
#combining inputs for buffer
data_a_f = data_A.flatten()
data_b_f = data_B.flatten()

data_input = np.concatenate([data_a_f, data_b_f])

In [ ]:
data_A

array([[4.35994902, 0.25926232, 5.49662478, ..., 6.61481327, 1.70477133,
        8.81652236],
       [7.7800816 , 1.33954208, 8.68916626, ..., 5.7027421 , 9.16714612,
        7.02264241],
       [5.01216452, 5.06086834, 2.18820786, ..., 4.3666988 , 0.15056134,
        8.92905883],
       ...,
       [8.47501085, 1.31358627, 7.48304695, ..., 6.13864418, 3.01381357,
        8.35725715],
       [5.90558724, 2.81559236, 4.71002749, ..., 6.55505965, 2.18741538,
        8.92183124],
       [7.20691133, 8.85091351, 7.88750645, ..., 7.34176132, 7.73163186,
        1.73273328]])

In [ ]:
data_a_f

array([4.35994902, 0.25926232, 5.49662478, ..., 7.34176132, 7.73163186,
       1.73273328])

In [ ]:
data_B

array([[0.45209358, 2.60834748, 8.56704315, ..., 2.37114778, 9.92721777,
        8.10776828],
       [1.94367587, 8.3472529 , 1.51141799, ..., 9.86539894, 2.7422922 ,
        3.71473337],
       [3.30308581, 3.61424772, 1.11340129, ..., 9.76225661, 8.75260893,
        8.83062626],
       ...,
       [4.59408327, 3.78022009, 2.15007284, ..., 5.41924595, 8.7485823 ,
        9.21298658],
       [1.15957171, 1.2703291 , 3.81098302, ..., 4.96930051, 3.38287824,
        6.98791335],
       [8.92869972, 5.85914667, 3.55248482, ..., 0.08168716, 8.70860167,
        0.38990867]])

In [ ]:
data_b_f

array([0.45209358, 2.60834748, 8.56704315, ..., 0.08168716, 8.70860167,
       0.38990867])

In [ ]:
data_input

array([4.35994902, 0.25926232, 5.49662478, ..., 0.08168716, 8.70860167,
       0.38990867])

In [ ]:
input_buffer = allocate(20000, np.float32)
output_buffer = allocate((100,100), np.float32)

In [ ]:
np.copyto(input_buffer, data_input)

In [ ]:
start_time = time.time()
data_send.transfer(input_buffer)
data_recv.transfer(output_buffer)
data_send.wait()
data_recv.wait()
end_time = time.time()
print(end_time - start_time)

0.11198782920837402


In [ ]:
output_buffer

PynqBuffer([[2260.2874, 2411.262 , 2386.4404, ..., 2321.1348, 2583.4348,
             2271.7097],
            [2406.3474, 2437.1956, 2404.0425, ..., 2404.8022, 2434.9695,
             2453.4414],
            [2656.1409, 2822.149 , 2811.0083, ..., 2754.5742, 2853.2485,
             2561.6997],
            ...,
            [2329.963 , 2418.727 , 2460.4373, ..., 2454.5156, 2746.152 ,
             2126.9338],
            [2398.1638, 2526.2656, 2517.2249, ..., 2372.3027, 2662.8118,
             2425.4265],
            [2579.4185, 2928.6133, 2757.6235, ..., 2538.9294, 2743.4634,
             2688.9077]], dtype=float32)

In [ ]:
input_buffer

PynqBuffer([4.359949  , 0.25926232, 5.496625  , ..., 0.08168715,
            8.708602  , 0.38990867], dtype=float32)

In [ ]:
diff = np.dot(data_A, data_B) - output_buffer

In [ ]:
rms_error = np.sqrt(np.mean(diff**2))

In [ ]:
diff

PynqBuffer([[-1.91465233e-06, -1.31885874e-04, -6.32932281e-04, ...,
              1.48166404e-04,  5.64897573e-04, -6.02358905e-05],
            [ 5.28194851e-05, -3.38601058e-05,  9.51904064e-04, ...,
              6.86210205e-06, -1.33828423e-04,  8.42856207e-05],
            [ 2.81803345e-04, -6.16184279e-04,  1.15561971e-03, ...,
             -4.36138338e-04,  3.12713758e-04,  2.59631734e-04],
            ...,
            [ 1.37445668e-04,  4.57161370e-04,  3.40686124e-04, ...,
             -1.29322588e-04,  3.20038692e-04,  2.46227785e-04],
            [-3.35393764e-04,  1.43864641e-04,  4.36434120e-04, ...,
             -6.93856191e-04,  2.80575089e-04,  9.67776191e-05],
            [ 1.61099002e-04, -5.04536051e-05,  2.17785204e-04, ...,
             -1.86180699e-04, -4.94276701e-04,  4.19518921e-04]])

In [ ]:
output_buffer

PynqBuffer([[2260.2874, 2411.262 , 2386.4404, ..., 2321.1348, 2583.4348,
             2271.7097],
            [2406.3474, 2437.1956, 2404.0425, ..., 2404.8022, 2434.9695,
             2453.4414],
            [2656.1409, 2822.149 , 2811.0083, ..., 2754.5742, 2853.2485,
             2561.6997],
            ...,
            [2329.963 , 2418.727 , 2460.4373, ..., 2454.5156, 2746.152 ,
             2126.9338],
            [2398.1638, 2526.2656, 2517.2249, ..., 2372.3027, 2662.8118,
             2425.4265],
            [2579.4185, 2928.6133, 2757.6235, ..., 2538.9294, 2743.4634,
             2688.9077]], dtype=float32)

In [ ]:
np.matmul(data_A, data_B)

array([[2260.2873516 , 2411.261831  , 2386.43979676, ..., 2321.13491379,
        2583.43537935, 2271.70965656],
       [2406.34746493, 2437.19552278, 2404.04343237, ..., 2404.80225296,
        2434.96934859, 2453.44149054],
       [2656.14115094, 2822.1483096 , 2811.0094564 , ..., 2754.57378261,
        2853.24884787, 2561.69996666],
       ...,
       [2329.96302807, 2418.72750794, 2460.43759655, ..., 2454.51549568,
        2746.15241965, 2126.93408412],
       [2398.16348297, 2526.26576886, 2517.22528995, ..., 2372.30204052,
        2662.81204815, 2425.42661045],
       [2579.41861813, 2928.6132308 , 2757.62375294, ..., 2538.92925718,
        2743.46288463, 2688.90813436]])

In [ ]:
diff

PynqBuffer([[-1.91465233e-06, -1.31885874e-04, -6.32932281e-04, ...,
              1.48166404e-04,  5.64897573e-04, -6.02358905e-05],
            [ 5.28194851e-05, -3.38601058e-05,  9.51904064e-04, ...,
              6.86210205e-06, -1.33828423e-04,  8.42856207e-05],
            [ 2.81803345e-04, -6.16184279e-04,  1.15561971e-03, ...,
             -4.36138338e-04,  3.12713758e-04,  2.59631734e-04],
            ...,
            [ 1.37445668e-04,  4.57161370e-04,  3.40686124e-04, ...,
             -1.29322588e-04,  3.20038692e-04,  2.46227785e-04],
            [-3.35393764e-04,  1.43864641e-04,  4.36434120e-04, ...,
             -6.93856191e-04,  2.80575089e-04,  9.67776191e-05],
            [ 1.61099002e-04, -5.04536051e-05,  2.17785204e-04, ...,
             -1.86180699e-04, -4.94276701e-04,  4.19518921e-04]])

In [ ]:
rms_error

PynqBuffer(0.00038044)